### dataset
1. IMDB
2. CNNNews
3. [PubMed](https://github.com/LIAAD/KeywordExtractor-Datasets/blob/master/datasets/PubMed.zip)

### preprocess
1. filter too frequent and less frequent words
2. stemming
3. document vector aggregation

### evaluation
1. F1
2. NDCG

In [1]:
import os
from collections import defaultdict
import math
import numpy as np 
import pandas as pd
import random
import re
import torch
import torch.nn as nn
from itertools import cycle
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
from tqdm.auto import tqdm

# Used to get the data
from sklearn.metrics import ndcg_score

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
nltk.download('stopwords')

import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use('Agg')

seed = 33

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chrisliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocess config

In [2]:
config = {}

config["dataset"] = "CNN" # "IMDB" "CNN", "PubMed"
config["n_document"] = 500
config["normalize_word_embedding"] = False
config["min_word_freq_threshold"] = 20
config["topk_word_freq_threshold"] = 100
config["document_vector_agg_weight"] = 'IDF' # ['mean', 'IDF', 'uniform', 'gaussian', 'exponential', 'pmi']
config["document_vector_weight_normalize"] = True # weighted sum or mean, True for mean, False for sum 
config["select_topk_TFIDF"] = None # ignore
config["embedding_file"] = "../data/glove.6B.100d.txt"
config["topk"] = [10, 30, 50]

In [3]:
def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    return True

In [4]:
def load_word2emb(embedding_file):
    word2embedding = dict()
    word_dim = int(re.findall(r".(\d+)d", embedding_file)[0])

    with open(embedding_file, "r") as f:
        for line in tqdm(f):
            line = line.strip().split()
            word = line[0]
            embedding = list(map(float, line[1:]))
            word2embedding[word] = np.array(embedding)

    print("Number of words:%d" % len(word2embedding))

    return word2embedding

word2embedding = load_word2emb(config["embedding_file"])

0it [00:00, ?it/s]

Number of words:400000


In [5]:
def normalize_wordemb(word2embedding):
    # Every word emb should have norm 1
    
    word_emb = []
    word_list = []
    for word, emb in word2embedding.items():
        word_list.append(word)
        word_emb.append(emb)

    word_emb = np.array(word_emb)

    for i in range(len(word_emb)):
        norm = np.linalg.norm(word_emb[i])
        word_emb[i] = word_emb[i] / norm

    for word, emb in tqdm(zip(word_list, word_emb)):
        word2embedding[word] = emb
    return word2embedding

if config["normalize_word_embedding"]:
    normalize_wordemb(word2embedding)

In [6]:
class Vocabulary:
    def __init__(self, word2embedding, config):
        # The low frequency words will be assigned as <UNK> token
        self.itos = {0: "<UNK>"}
        self.stoi = {"<UNK>": 0}
        
        self.word2embedding = word2embedding
        self.config = config

        self.word_freq_in_corpus = defaultdict(int)
        self.IDF = {}
        self.ps = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        
        self.word_dim = len(word2embedding['the'])
    def __len__(self):
        return len(self.itos)

    def tokenizer_eng(self, text):
        text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
        text = text.strip().split()
        
        return [self.ps.stem(w) for w in text if w.lower() not in self.stop_words]
    
    def read_raw(self):        
        if self.config["dataset"] == 'IMDB':
            data_file_path = '../data/IMDB.txt'
        elif self.config["dataset"] == 'CNN':
            data_file_path = '../data/CNN.txt'
        elif self.config["dataset"] == 'PubMed':
            data_file_path = '../data/PubMed.txt'
        
        # raw documents
        self.raw_documents = []
        with open(data_file_path,'r',encoding='utf-8') as f:
            for line in tqdm(f, desc="Loading documents"):
                self.raw_documents.append(line.strip("\n"))
                
        return self.raw_documents
    
    def build_vocabulary(self):
        sentence_list = self.raw_documents
        
        self.doc_freq = defaultdict(int) # # of document a word appear
        self.document_num = len(sentence_list)
        self.word_vectors = [[0]*self.word_dim] # unknown word emb
        
        for sentence in tqdm(sentence_list, desc="Preprocessing documents"):
            # for doc_freq
            document_words = set()
            
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.word2embedding:
                    continue
                    
                # calculate word freq
                self.word_freq_in_corpus[word] += 1
                document_words.add(word)
                
            for word in document_words:
                self.doc_freq[word] += 1
        
        # calculate IDF
        print('doc num', self.document_num)
        for word, freq in self.doc_freq.items():
            self.IDF[word] = math.log(self.document_num / (freq+1))
        
        # delete less freq words:
        delete_words = []
        for word, v in self.word_freq_in_corpus.items():
            if v < self.config["min_word_freq_threshold"]:
                delete_words.append(word)     
        for word in delete_words:
            del self.IDF[word]    
            del self.word_freq_in_corpus[word]    
        
        # delete too freq words
        print('eliminate freq words')
        IDF = [(word, freq) for word, freq in self.IDF.items()]
        IDF.sort(key=lambda x: x[1])

        for i in range(self.config["topk_word_freq_threshold"]):
            print(word)
            word = IDF[i][0]
            del self.IDF[word]
            del self.word_freq_in_corpus[word]
        
        # construct word_vectors
        idx = 1
        for word in self.word_freq_in_corpus:
            self.word_vectors.append(self.word2embedding[word])
            self.stoi[word] = idx
            self.itos[idx] = word
            idx += 1
            
    def init_word_weight(self,sentence_list, agg):
        if agg == 'mean':
            self.word_weight = {word: 1 for word in self.IDF.keys()}
        elif agg == 'IDF':
            self.word_weight = self.IDF
        elif agg == 'uniform':
            self.word_weight = {word: np.random.uniform(low=0.0, high=1.0) for word in self.IDF.keys()}
        elif agg == 'gaussian':
            mu, sigma = 10, 1 # mean and standard deviation
            self.word_weight = {word: np.random.normal(mu, sigma) for word in self.IDF.keys()}
        elif agg == 'exponential':
            self.word_weight = {word: np.random.exponential(scale=1.0) for word in self.IDF.keys()}
        elif agg == 'pmi':
            trigram_measures = BigramAssocMeasures()
            self.word_weight = defaultdict(int)
            corpus = []

            for text in tqdm(sentence_list):
                corpus.extend(text.split())

            finder = BigramCollocationFinder.from_words(corpus)
            for pmi_score in finder.score_ngrams(trigram_measures.pmi):
                pair, score = pmi_score
                self.word_weight[pair[0]] += score
                self.word_weight[pair[1]] += score
                
    def calculate_document_vector(self):
        # Return
        # document_vectors: weighted sum of word emb
        # document_answers_idx: doc to word index list
        # document_answers_wsum: word weight summation, e.g. total TFIDF score of a doc
        
        document_vectors = [] 
        document_answers = []
        document_answers_wsum = []
        
        sentence_list = self.raw_documents
        agg = self.config["document_vector_agg_weight"]
        n_document = self.config["n_document"]
        select_topk_TFIDF = self.config["select_topk_TFIDF"]
        
        self.init_word_weight(sentence_list, agg)
        for sentence in tqdm(sentence_list[:min(n_document, len(sentence_list))], desc="calculate document vectors"):
            document_vector = np.zeros(len(self.word_vectors[0]))
            select_words = []
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.stoi:
                    continue
                else:
                    select_words.append(word)

            # select topk TDIDF
            if select_topk_TFIDF is not None:
                doc_TFIDF = defaultdict(float)
                for word in select_words:    
                    doc_TFIDF[word] += self.IDF[word]

                doc_TFIDF_l = [(word, TFIDF) for word, TFIDF in doc_TFIDF.items()]
                doc_TFIDF_l.sort(key=lambda x:x[1], reverse=True)
                
                select_topk_words = set(list(map(lambda x:x[0], doc_TFIDF_l[:select_topk_TFIDF])))
                select_words = [word for word in select_words if word in select_topk_words]
            else:
                pass
            
            total_weight = 0
            # aggregate to doc vectors
            for word in select_words:
                document_vector += np.array(self.word2embedding[word]) * self.word_weight[word]
                total_weight += self.word_weight[word]
                
            if len(select_words) == 0:
                print('error', sentence)
                continue
            else:
                if self.config["document_vector_weight_normalize"]:
                    document_vector /= total_weight
                    total_weight = 1
            
            document_vectors.append(document_vector)
            document_answers.append(select_words)
            document_answers_wsum.append(total_weight)
        
        # get answers
        document_answers_idx = []    
        for ans in document_answers:
            ans_idx = []
            for token in ans:
                if token in self.stoi:
                    ans_idx.append(self.stoi[token])                    
            document_answers_idx.append(ans_idx)
        
        self.document_vectors = document_vectors
        self.document_answers_idx = document_answers_idx
        self.document_answers_wsum = document_answers_wsum
        
        return document_vectors, document_answers_idx, document_answers_wsum
        
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]
    
    def check_docemb(self):
        word_vectors = np.array(self.word_vectors)
        pred = np.zeros(word_vectors.shape[1])
        cnt = 0

        for word_idx in self.document_answers_idx[0]:
            pred += word_vectors[word_idx] * self.word_weight[self.itos[word_idx]]
            cnt += self.word_weight[self.itos[word_idx]]
        
        if self.config["document_vector_weight_normalize"]:
            pred /= cnt
        assert np.sum(self.document_vectors[0]) - np.sum(pred) == 0

In [7]:
def build_vocab(config, word2embedding):
    # build vocabulary
    vocab = Vocabulary(word2embedding, config)
    vocab.read_raw()
    vocab.build_vocabulary()
    vocab_size = len(vocab)
    # get doc emb
    vocab.calculate_document_vector()
    vocab.check_docemb()
    
    return vocab

vocab = build_vocab(config, word2embedding)

Loading documents: 0it [00:00, ?it/s]

Preprocessing documents:   0%|          | 0/19026 [00:00<?, ?it/s]

doc num 19026
eliminate freq words
paroxysm
subject
line
organ
write
univers
one
would
use
like
get
know
dont
think
time
make
also
say
go
im
could
want
new
work
good
well
way
need
look
even
anyon
thing
see
tri
thank
much
year
world
system
right
problem
may
take
mani
two
first
seem
question
pleas
1
state
us
come
2
post
help
call
usa
point
sinc
find
read
still
back
mean
ive
give
email
sure
differ
might
run
cant
reason
last
day
interest
case
let
person
said
never
start
doesnt
tell
better
ask
got
without
follow
part
lot
3
number
put
fact
gener
inform
actual
that


calculate document vectors:   0%|          | 0/500 [00:00<?, ?it/s]

In [8]:
print("Finish building dataset!")
print(f"Number of documents:{len(vocab.raw_documents)}")
print(f"Number of words:{len(vocab)}")

l = list(map(len, vocab.document_answers_idx))
print("Average length of document:", np.mean(l))

Finish building dataset!
Number of documents:19026
Number of words:7602
Average length of document: 73.02


In [9]:
word_vectors = np.array(vocab.word_vectors)
print("word_vectors:", word_vectors.shape)

document_vectors = np.array(vocab.document_vectors)
print("document_vectors", document_vectors.shape)

document_answers_wsum = np.array(vocab.document_answers_wsum).reshape(-1, 1)
print("document_answers_wsum", document_answers_wsum.shape)

# create weight_ans
document_answers_idx = vocab.document_answers_idx

# random shuffle
shuffle_idx = list(range(len(document_vectors)))
random.Random(seed).shuffle(shuffle_idx)

document_vectors = document_vectors[shuffle_idx]
document_answers_wsum = document_answers_wsum[shuffle_idx]
document_answers_idx = [document_answers_idx[idx] for idx in shuffle_idx]

word_vectors: (7602, 100)
document_vectors (500, 100)
document_answers_wsum (500, 1)


In [10]:
# onthot_ans: word freq matrix
# weight_ans: TFIDF matrix

onehot_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
weight_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
print(weight_ans.shape)

for i in tqdm(range(len(document_answers_idx))):
    for word_idx in document_answers_idx[i]:
        weight_ans[i, word_idx] += vocab.word_weight[vocab.itos[word_idx]]
        onehot_ans[i, word_idx] += 1
        
    if config["document_vector_weight_normalize"]:
        weight_ans[i] /= np.sum(weight_ans[i])

(500, 7602)


  0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
# check
assert np.sum(document_vectors - np.dot(weight_ans, word_vectors) > 1e-10) == 0

## Results

In [12]:
final_results = []
select_columns = ['model']
for topk in config["topk"]:
    select_columns.append('percision@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('recall@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('F1@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('ndcg@{}'.format(topk))
select_columns.append('ndcg@all')
select_columns

['model',
 'percision@10',
 'percision@30',
 'percision@50',
 'recall@10',
 'recall@30',
 'recall@50',
 'F1@10',
 'F1@30',
 'F1@50',
 'ndcg@10',
 'ndcg@30',
 'ndcg@50',
 'ndcg@all']

## setting training size

In [13]:
train_size_ratio = 1
train_size = int(len(document_answers_idx) * train_size_ratio)
train_size

500

## Top K freq word

In [14]:
topk_results = {}

In [15]:
test_ans = document_answers_idx[:train_size]

In [16]:
word_freq = [(word, freq) for word, freq in vocab.word_freq_in_corpus.items()]
word_freq.sort(key=lambda x:x[1], reverse=True)
word_freq[:10]

[('x', 6539),
 ('god', 5208),
 ('file', 4918),
 ('0', 4520),
 ('window', 4444),
 ('program', 4201),
 ('drive', 3633),
 ('4', 3528),
 ('game', 3474),
 ('govern', 3268)]

In [17]:
def topk_word_evaluation(k=50):
    topk_word = [word for (word, freq) in word_freq[:k]]

    pr, re = [], []
    for ans in tqdm(test_ans):
        ans = set(ans)
        ans = [vocab.itos[a] for a in ans]

        hit = []
        for word in ans:
            if word in topk_word:
                hit.append(word)

        precision = len(hit) / k
        recall = len(hit) / len(ans)
        pr.append(precision)
        re.append(recall)

    pr = np.mean(pr)
    re = np.mean(re)
    f1 = 2 * pr * re / (pr + re) if (pr + re) != 0 else 0
    print('top {} word'.format(k))
    print('percision', np.mean(pr))
    print('recall', np.mean(re))
    print('F1', f1)
    return f1


for topk in config['topk']:
    topk_results["F1@{}".format(topk)] = topk_word_evaluation(k=topk)


  0%|          | 0/500 [00:00<?, ?it/s]

top 10 word
percision 0.07180000000000002
recall 0.016975938403991114
F1 0.02745952111167467


  0%|          | 0/500 [00:00<?, ?it/s]

top 30 word
percision 0.07719999999999999
recall 0.05033731696316411
F1 0.06093966788839775


  0%|          | 0/500 [00:00<?, ?it/s]

top 50 word
percision 0.07504000000000001
recall 0.07963191243187574
F1 0.07726779367934514


In [18]:
def topk_word_evaluation_NDCG(k=50):
    freq_word =[word for (word, freq) in word_freq]
    freq_word_idx = [vocab.stoi[word] for word in freq_word if word in vocab.stoi]
    
    scores = np.zeros(len(vocab.word_vectors))
    for rank, idx in enumerate(freq_word_idx):
        scores[idx] = len(vocab.word_vectors) - rank
    
    NDCGs = []
    
    for ans in tqdm(test_ans):
        weight_ans = np.zeros(len(vocab.word_vectors))
        
        for word_idx in ans:
            if word_idx == 0:
                continue
            word = vocab.itos[word_idx]
            weight_ans[word_idx] += vocab.IDF[word]

        NDCG_score = ndcg_score(weight_ans.reshape(1,-1), scores.reshape(1,-1), k=k)
        NDCGs.append(NDCG_score)

    print('top {} NDCG:{}'.format(k, np.mean(NDCGs)))
    
    return np.mean(NDCGs)


# for topk in config['topk']:
#     topk_results["ndcg@{}".format(topk)] = topk_word_evaluation_NDCG(k=topk)
    
# topk_results["ndcg@all"] = topk_word_evaluation_NDCG(k=None)


In [19]:
topk_results["model"] = "topk"
final_results.append(pd.Series(topk_results))

## Sklearn

In [20]:
from sklearn.linear_model import LinearRegression, Lasso

In [21]:
print(document_vectors.shape)
print(weight_ans.shape)
print(word_vectors.shape)

(500, 100)
(500, 7602)
(7602, 100)


In [22]:
def evaluate_sklearn(pred, ans):
    results = {}
        
    one_hot_ans = np.arange(ans.shape[0])[ans > 0]
    
    for topk in config["topk"]:
        one_hot_pred = np.argsort(pred)[-topk:]
        hit = np.intersect1d(one_hot_pred, one_hot_ans)
        percision = len(hit) / topk
        recall = len(hit) / len(one_hot_ans)
        f1 = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
        
        results['percision@{}'.format(topk)] = percision
        results['recall@{}'.format(topk)] = recall
        results['F1@{}'.format(topk)] = f1
        
    ans = ans.reshape(1, -1)
    pred = pred.reshape(1, -1)
    for topk in config["topk"]:
        results['ndcg@{}'.format(topk)] = ndcg_score(ans, pred, k=topk)

    results['ndcg@all'] = (ndcg_score(ans, pred, k=None))
    
    return results

### linear regression

In [23]:
results = []

for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
    x = word_vectors.T
    y = doc_emb
    
    ans = weight_ans[doc_id]
    model = LinearRegression(fit_intercept=False).fit(x, y)
    r2 = model.score(x, y)

    res = evaluate_sklearn(model.coef_, ans)
    results.append(res)

  0%|          | 0/500 [00:00<?, ?it/s]

In [24]:
results = pd.DataFrame(results).mean()
results['model'] = 'sk-linear-regression'
final_results.append(results)
results

percision@10                   0.391
recall@10                    0.11406
F1@10                       0.159542
percision@30                  0.2262
recall@30                   0.184811
F1@30                       0.179824
percision@50                 0.17128
recall@50                   0.224501
F1@50                       0.172255
ndcg@10                     0.483467
ndcg@30                     0.432654
ndcg@50                     0.430092
ndcg@all                    0.623725
model           sk-linear-regression
dtype: object

### lasso

In [25]:
results = []
sk_lasso_epoch = 1000

for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
    x = word_vectors.T
    y = doc_emb
    
    ans = weight_ans[doc_id]
    model = Lasso(positive=True, fit_intercept=False, alpha=0.0001, max_iter=sk_lasso_epoch, tol=0).fit(x, y)
    r2 = model.score(x, y)

    res = evaluate_sklearn(model.coef_, ans)
    results.append(res)

  0%|          | 0/500 [00:00<?, ?it/s]

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001724544758088399, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0019690437566093075, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0044089386910606265, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increas

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017295447782276184, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0003405292732725047, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017349524060469405, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increa

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001819887918682427, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0022703314108754456, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001211160172059817, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017655533509107683, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0010065671291614995, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0008457014809340937, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increa

...

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001462718667739385, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0018051092530687578, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0011742406679428876, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increas

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0004026483107275819, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0011683298663542824, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0015759714039317266, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increa

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0003276551020927, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0012930539330719703, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0018825623053839854, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase 

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0015882535394630406, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0022908211691602793, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0018695535359306568, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increa

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0014312599298903202, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0016037291484068043, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0010539081188959817, tolerance: 0.0
  positive)
/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increa

In [26]:
results = pd.DataFrame(results).mean()
results['model'] = 'sk-lasso'
final_results.append(results)
results

percision@10      0.6266
recall@10       0.208416
F1@10           0.284994
percision@30    0.349333
recall@30        0.31626
F1@30           0.296616
percision@50     0.25168
recall@50       0.361018
F1@50           0.265201
ndcg@10         0.684684
ndcg@30         0.602965
ndcg@50         0.593951
ndcg@all        0.726974
model           sk-lasso
dtype: object

## Our Model

In [27]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [28]:
class Custom_Lasso_Dataset(Dataset):
    def __init__(self, 
                 doc_vectors,
                 doc_w_sum,
                 weight_ans
                 ):
        self.doc_vectors = torch.FloatTensor(doc_vectors)
        self.doc_w_sum = torch.FloatTensor(doc_w_sum)
        self.weight_ans = weight_ans
        assert len(doc_vectors) == len(doc_w_sum)
        
    def __getitem__(self, idx):
                
        return self.doc_vectors[idx], self.doc_w_sum[idx], idx

    def __len__(self):
        return len(self.doc_vectors)


In [29]:
class LR(nn.Module):
    """
    Input shape: (N, 3, 64, 64)
    Output shape: (N, )
    """
    def __init__(self, num_doc, num_words):
        super(LR, self).__init__()
        weight = torch.zeros(num_doc, num_words).to(device)
        self.emb = torch.nn.Embedding.from_pretrained(weight, freeze=False)
        
    def forward(self, doc_ids, word_vectors):
        return self.emb(doc_ids) @ word_vectors

In [30]:
def evaluate_Custom_Lasso(model, train_loader):
    results = {}
    model.eval()
    
    scores = np.array(model.emb.cpu().weight.data)
    model.emb.to(device)
    true_relevance = train_loader.dataset.weight_ans

    # F1
    F1s = []
    precisions = []
    recalls = []
    for i in range(true_relevance.shape[0]):
        one_hot_ans = np.arange(true_relevance.shape[1])[true_relevance[i] > 0]
        pred = scores[i]
        
        F1_ = []
        percision_ = []
        recall_ = []
        for topk in config["topk"]:
            one_hot_pred = np.argsort(pred)[-topk:]
            
            hit = np.intersect1d(one_hot_pred, one_hot_ans)
            percision = len(hit) / topk
            recall = len(hit) / len(one_hot_ans)
            
            F1 = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
            F1_.append(F1)
            percision_.append(percision)
            recall_.append(recall)
            
        F1s.append(F1_)
        precisions.append(percision_)
        recalls.append(recall_)
        
    F1s = np.mean(F1s, axis=0)
    precisions = np.mean(precisions, axis=0)
    recalls = np.mean(recalls, axis=0)
    
    for i, topk in enumerate(config["topk"]):
        results['F1@{}'.format(topk)] = F1s[i]
        results['percision@{}'.format(topk)] = precisions[i]
        results['recall@{}'.format(topk)] = recalls[i]

    # NDCG
    for topk in config["topk"]:
        results['ndcg@{}'.format(topk)] = ndcg_score(true_relevance, scores, k=topk)
    results['ndcg@all'] = ndcg_score(true_relevance, scores, k=None)
    
    return results

In [31]:
batch_size = 100
print('document num', train_size)

train_dataset = Custom_Lasso_Dataset(document_vectors[:train_size], document_answers_wsum[:train_size], weight_ans[:train_size])
train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

document num 500


## start training

In [32]:
def train_CLasso(select_idx):
    batch_size = len(select_idx)
    print('batch_size', batch_size)
    train_dataset = Custom_Lasso_Dataset(document_vectors[select_idx], document_answers_wsum[select_idx], weight_ans[select_idx])
    train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    
    model = LR(num_doc=batch_size, num_words=word_vectors.shape[0]).to(device)
    model.train()

    word_vectors_tensor = torch.FloatTensor(word_vectors).to(device)

    opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
    criterion = nn.MSELoss(reduction='mean')

    results = []
    step = 0
    for epoch in tqdm(range(n_epoch)):    
        loss_mse_his = []
        loss_w_reg_his = []

        model.train()

        for data in train_loader:
            doc_embs, doc_w_sum, doc_ids = data

            doc_embs = doc_embs.to(device)
            doc_w_sum = doc_w_sum.to(device)
            doc_ids = doc_ids.to(device)

            w_reg = doc_w_sum * w_sum_reg_mul
            # w_reg = (torch.ones(doc_embs.size(0), 1) * w_sum_reg_mul).to(device)

            # MSE loss
            pred_doc_embs = model(doc_ids, word_vectors_tensor)     
            loss_mse = criterion(pred_doc_embs, doc_embs)

            pred_w_sum = torch.sum(model.emb(doc_ids), axis=1).view(-1, 1)
            loss_w_reg = criterion(pred_w_sum, w_reg)

            loss_l1 = torch.sum(torch.abs(model.emb(doc_ids)))
            loss = loss_mse + loss_w_reg * w_sum_reg + loss_l1 * L1

            # Model backwarding
            model.zero_grad()
            loss.backward()
            opt.step()

            loss_mse_his.append(loss_mse.item())
            loss_w_reg_his.append(loss_w_reg.item())

            for p in model.parameters():
                p.data.clamp_(w_clip_value, float('inf'))


        if (verbose and epoch % valid_epoch == 0) or (not verbose and epoch == n_epoch-1):
            res = {}
            res['epoch'] = epoch
            res['loss_mse'] = np.mean(loss_mse_his)
            res['loss_w_reg'] = np.mean(loss_w_reg_his)

            res_ndcg = evaluate_Custom_Lasso(model, train_loader)
            res.update(res_ndcg)
            results.append(res)

            print()
            for k, v in res.items():
                print(k, v)
                
    return res, model

# setting
lr = 1
momentum = 0.999
weight_decay = 0
nesterov = False # True

n_epoch = 1000

w_sum_reg = 1e-2
w_sum_reg_mul = 1
w_clip_value = 0

L1 = 1e-5

verbose = False
valid_epoch = 100

results = []
for i in range(len(document_vectors)//batch_size+1):
    if i != len(document_vectors)//batch_size:
        select_idx = np.arange(i*batch_size, (i+1)*batch_size)
    elif len(document_vectors)%batch_size != 0:
        select_idx = np.arange(i*batch_size, len(document_vectors))
    else:
        break
    res, model = train_CLasso(select_idx)
    results.append(res)

batch_size 100


  0%|          | 0/1000 [00:00<?, ?it/s]


epoch 999
loss_mse 0.01485817227512598
loss_w_reg 0.22490563988685608
F1@10 0.2951997356997321
percision@10 0.665
recall@10 0.22696172922626068
F1@30 0.2899589582358517
percision@30 0.34733333333333327
recall@30 0.3213212287707476
F1@50 0.2550103550163377
percision@50 0.24400000000000005
recall@50 0.36021490350018104
ndcg@10 0.7619119347401595
ndcg@30 0.6598783010647099
ndcg@50 0.6417756085147643
ndcg@all 0.778317477087374
batch_size 100


  0%|          | 0/1000 [00:00<?, ?it/s]


epoch 999
loss_mse 0.013244658708572388
loss_w_reg 0.20534354448318481
F1@10 0.2789122507411225
percision@10 0.6389999999999999
recall@10 0.20294492881391993
F1@30 0.29278019956362333
percision@30 0.3566666666666666
recall@30 0.30630505887344456
F1@50 0.2544321958666419
percision@50 0.2458
recall@50 0.3402695068131304
ndcg@10 0.7263056373151484
ndcg@30 0.6301800738451833
ndcg@50 0.610736354852434
ndcg@all 0.7586522323476943
batch_size 100


...


epoch 999
loss_mse 0.009370246902108192
loss_w_reg 0.15271569788455963
F1@10 0.29788659424024844
percision@10 0.6639999999999999
recall@10 0.21865689293568025
F1@30 0.3001944126711866
percision@30 0.3626666666666665
recall@30 0.32105658005159476
F1@50 0.2649429056195579
percision@50 0.25800000000000006
recall@50 0.35889937458914367
ndcg@10 0.7424771988448141
ndcg@30 0.6476421101649253
ndcg@50 0.6284649578660351
ndcg@all 0.7703777016357855
batch_size 100


  0%|          | 0/1000 [00:00<?, ?it/s]


epoch 999
loss_mse 0.009284747764468193
loss_w_reg 0.1408132165670395
F1@10 0.341107053313821
percision@10 0.6699999999999998
recall@10 0.2523101428184931
F1@30 0.3144441964124096
percision@30 0.34133333333333327
recall@30 0.35312837702424915
F1@50 0.26961503671684434
percision@50 0.2376
recall@50 0.39601606608414075
ndcg@10 0.7363901435631237
ndcg@30 0.6355688184415278
ndcg@50 0.6250805761381819
ndcg@all 0.7685588035708022
batch_size 100


  0%|          | 0/1000 [00:00<?, ?it/s]


epoch 999
loss_mse 0.010696711018681526
loss_w_reg 0.17409618198871613
F1@10 0.295100688875262
percision@10 0.6769999999999999
recall@10 0.2037612230818682
F1@30 0.30263127071349377
percision@30 0.35366666666666674
recall@30 0.3077203062935007
F1@50 0.2631711486336721
percision@50 0.24479999999999996
recall@50 0.343731088680005
ndcg@10 0.7355931216276379
ndcg@30 0.6305553137793638
ndcg@50 0.6129524283392148
ndcg@all 0.7596966401104247


In [33]:
pd.set_option('display.max_rows', 500)
results_df = pd.DataFrame(results).set_index('epoch').mean()
results_df

loss_mse        0.011491
loss_w_reg      0.179575
F1@10           0.301641
percision@10    0.663000
recall@10       0.220927
F1@30           0.300002
percision@30    0.352333
recall@30       0.321906
F1@50           0.261434
percision@50    0.246040
recall@50       0.359826
ndcg@10         0.740536
ndcg@30         0.640765
ndcg@50         0.623802
ndcg@all        0.767121
dtype: float64

In [34]:
results_df['model'] = 'our-lasso'
final_results.append(results_df[select_columns])

## Quality Check

In [35]:
# select doc_id and k
doc_id = 40
topk = 30

model

LR(
  (emb): Embedding(100, 7602)
)

In [36]:
import colored
from colored import stylize

word_list = vocab.itos

gt = [word_list[word_idx] for word_idx in np.argsort(weight_ans[doc_id])[::-1][:topk]]
pred = [word_list[word_idx] for word_idx in np.argsort(model.emb.cpu().weight.data[doc_id].numpy())[::-1][:topk]]

print('ground truth')
for word in gt:
    if word in pred:
        print(stylize(word, colored.bg("yellow")), end=' ')
    else:
        print(word, end=' ')

print()
print('\nprediction')
for word in pred:
    if word in gt:
        print(stylize(word, colored.bg("yellow")), end=' ')
    else:
        print(word, end=' ')


ground truth
fear gun hike afraid trail base threaten threat sound andi backpack firsthand mindset rude shelter analyst lend lone sort safer paranoia effect inner feel your freeman other slight 77 paranoid 

prediction
vga vesa jc byte 640x480 256 mode tsr id punch svga ega reward stat ad et chip khz gd ballot pixel color yr hdtv nt raw video password 2d rpm 

In [37]:
# raw document
print()
ps = PorterStemmer()
    
for word in vocab.raw_documents[doc_id].split():
    word_stem = ps.stem(word).lower()

    if word_stem in gt:
        if word_stem in pred:
            print(stylize(word, colored.bg("yellow")), end=' ')
        else:
            print(stylize(word, colored.bg("light_gray")), end=' ')
    else:
        print(word, end=' ')
# print(dataset.documents[doc_id])


From: keith@cco.caltech.edu (Keith Allan Schneider) Subject: Re: The Universe and Black Holes, was Re: 2000 years..... Organization: California Institute of Technology, Pasadena Lines: 23 NNTP-Posting-Host: punisher.caltech.edu salem@pangea.Stanford.EDU (Bruce Salem) writes: >Is this Zeno's Paradox? No. Zeno's paradox is resolved by showing that integration or an infinite series of decreasing terms can sum to a finite result. >Nothing can get out of a black hole because >the escape velocity is the speed of light. I don't know how time dialation >can prevent matter spiraling in from getting to the event horizon. Can any- >one explain how matter gets in. Well, suppose a probe emitting radiation at a constant frequency was sent towards a black hole. As it got closer to the event horizon, the red shift would keep increasing. The period would get longer and longer, but it would never stop. An observer would not observe the probe actually reaching the event horizon. The detected energy from

In [38]:
results = {}
   
scores = np.array(model.emb.weight.data)[doc_id].reshape(1, -1)
true_relevance = train_loader.dataset.weight_ans[doc_id].reshape(1, -1)

results['ndcg@50'] = (ndcg_score(true_relevance, scores, k=50))
results['ndcg@100'] = (ndcg_score(true_relevance, scores, k=100))
results['ndcg@200'] = (ndcg_score(true_relevance, scores, k=200))
results['ndcg@all'] = (ndcg_score(true_relevance, scores, k=None))

print('This document ndcg:')
print('ground truth length:', np.sum(weight_ans[doc_id] > 0))
print('NDCG top50', results['ndcg@50'])
print('NDCG top100', results['ndcg@100'])
print('NDCG top200', results['ndcg@200'])
print('NDCG ALL', results['ndcg@all'])


This document ndcg:
ground truth length: 117
NDCG top50 0.0
NDCG top100 0.014873665453818127
NDCG top200 0.03280090342747383
NDCG ALL 0.3693629103368511


## Final results

In [39]:
is_notebook = in_notebook()

In [40]:
final_results_df = pd.DataFrame(final_results).reset_index(drop=True)

experiment_dir = './records/dataset-{}-n_document-{}-wdist-{}-filtertopk-{}'.format(
                                        config['dataset'],
                                        config['n_document'],
                                        config["document_vector_agg_weight"],
                                        config["topk_word_freq_threshold"])

print('Saving to directory', experiment_dir)
os.makedirs(experiment_dir, exist_ok=True)

Saving to directory ./records/dataset-CNN-n_document-500-wdist-IDF-filtertopk-100


In [41]:
final_results_df.to_csv(os.path.join(experiment_dir, 'result.csv'), index=False)

import json
with open(os.path.join(experiment_dir, 'config.json'), 'w') as json_file:
    json.dump(config, json_file)

In [42]:
for feat in final_results_df.set_index('model').columns:
    plt.bar(final_results_df['model'],
            final_results_df[feat], 
            width=0.5, 
            bottom=None, 
            align='center', 
            color=['lightsteelblue', 
                   'cornflowerblue', 
                   'royalblue', 
                   'navy'])
    plt.title(feat)
    plt.savefig(os.path.join(experiment_dir, '{}.png'.format(feat)))
    plt.clf()
    if is_notebook:
        plt.show()

/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  from ipykernel import kernelapp as app


In [43]:
print(final_results_df)
final_results_df

      F1@10     F1@30     F1@50                 model  percision@10  \
0  0.027460  0.060940  0.077268                  topk           NaN   
1  0.159542  0.179824  0.172255  sk-linear-regression        0.3910   
2  0.284994  0.296616  0.265201              sk-lasso        0.6266   
3  0.301641  0.300002  0.261434             our-lasso        0.6630   

   recall@10  percision@30  recall@30  percision@50  recall@50   ndcg@10  \
0        NaN           NaN        NaN           NaN        NaN       NaN   
1   0.114060      0.226200   0.184811       0.17128   0.224501  0.483467   
2   0.208416      0.349333   0.316260       0.25168   0.361018  0.684684   
3   0.220927      0.352333   0.321906       0.24604   0.359826  0.740536   

    ndcg@30   ndcg@50  ndcg@all  
0       NaN       NaN       NaN  
1  0.432654  0.430092  0.623725  
2  0.602965  0.593951  0.726974  
3  0.640765  0.623802  0.767121  


,F1@10,F1@30,F1@50,model,percision@10,recall@10,percision@30,recall@30,percision@50,recall@50,ndcg@10,ndcg@30,ndcg@50,ndcg@all
0,0.027460,0.060940,0.077268,topk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.159542,0.179824,0.172255,sk-linear-regression,0.3910,0.114060,0.226200,0.184811,0.17128,0.224501,0.483467,0.432654,0.430092,0.623725
2,0.284994,0.296616,0.265201,sk-lasso,0.6266,0.208416,0.349333,0.316260,0.25168,0.361018,0.684684,0.602965,0.593951,0.726974
3,0.301641,0.300002,0.261434,our-lasso,0.6630,0.220927,0.352333,0.321906,0.24604,0.359826,0.740536,0.640765,0.623802,0.767121
